In [ ]:
# --- Object Detection using Transfer Learning (Offline Version) ---
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

# --- Load CIFAR-10 Dataset (Offline) ---
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # normalize
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# --- Load Pretrained VGG16 (Transfer Learning) ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# --- Freeze lower convolutional layers ---
for layer in base_model.layers:
    layer.trainable = False

# --- Add custom classifier ---
model = Sequential([
    base_model,
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# --- Compile and train ---
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# --- Optional fine-tuning ---
for layer in base_model.layers[-4:]:
    layer.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_test, y_test))


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 831s 1s/step - accuracy: 0.4004 - loss: 1.7058 - val_accuracy: 0.5470 - val_loss: 1.2930
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 825s 1s/step - accuracy: 0.5419 - loss: 1.3185 - val_accuracy: 0.5748 - val_loss: 1.2188
Epoch 3/5
680/782 ━━━━━━━━━━━━━━━━━━━━ 1:28 872ms/step - accuracy: 0.5607 - loss: 1.2459